<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [3]:
import folium
import wget
import pandas as pd

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


In [9]:
## Task 1: Mark all launch sites on a map

# Example DataFrame (replace with your real data if available)
launch_sites = {
    'Launch_Site': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
    'Lat': [28.5623, 34.6321, 28.5733, 28.5632],
    'Long': [-80.5774, -120.6108, -80.6469, -80.5768]
}

df = pd.DataFrame(launch_sites)

# Step 1: Create a folium Map centered around Florida (or your center)
map = folium.Map(location=[28.5623, -80.5774], zoom_start=5)

# Step 2: Add Markers for each Launch Site
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch_Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(map)

# Step 3: Display the map
map


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [11]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [13]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [15]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [17]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [19]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [23]:
# Task 2: Mark the success/failed launches for each site on the map
# Import packages
import folium
import pandas as pd

# Example DataFrame (you should replace it with your actual SpaceX data)
# Let's assume you have a DataFrame 'df' with Launch Site, Latitude, Longitude, and Class (Success/Fail)
launch_data = {
    'Launch_Site': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
    'Lat': [28.5623, 34.6321, 28.5733, 28.5632],
    'Long': [-80.5774, -120.6108, -80.6469, -80.5768],
    'Class': [1, 0, 1, 0]   # 1 = Success, 0 = Failure
}

df = pd.DataFrame(launch_data)

# Step 1: Create the base map
launch_map = folium.Map(location=[28.5623, -80.5774], zoom_start=5)

# Step 2: Define a function to choose the marker color based on 'Class'
def get_marker_color(launch_success):
    if launch_success == 1:
        return 'green'
    else:
        return 'red'

# Step 3: Add Markers for each Launch
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"Launch Site: {row['Launch_Site']}<br>Outcome: {'Success' if row['Class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=get_marker_color(row['Class']), icon='rocket', prefix='fa')
    ).add_to(launch_map)

# Step 4: Display the map
launch_map


In [21]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [27]:
marker_cluster = MarkerCluster()
# Create a new column 'marker_color' based on the 'class' value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [29]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
print(spacex_df.columns.tolist())

['Launch Site', 'Lat', 'Long', 'class', 'marker_color']


In [31]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [35]:
# 2. Create and add a MarkerCluster to the site map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

# 3. For each launch, create a marker with correct color
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [41]:
!pip install geopy

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   -------------------------- ------------- 81.9/125.4 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00


In [43]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Import the distance module from geopy
from geopy.distance import geodesic

# Example: Let's say you have the coordinates of the launch site
launch_site_coords = (28.562302, -80.577356)  # Example for CCAFS LC-40

# And a point of proximity (like an airport or city center)
proximity_coords = (28.572872, -80.648980)  # Example for a nearby landmark

# Calculate the distance between the launch site and the proximity
distance_km = geodesic(launch_site_coords, proximity_coords).km

print(f"The distance between the launch site and the proximity is {distance_km:.2f} km")


The distance between the launch site and the proximity is 7.11 km


In [45]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map

# Import MousePosition from folium.plugins
from folium.plugins import MousePosition

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [49]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [51]:
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance using the Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Approximate radius of the Earth in kilometers

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Coordinates of the launch site (example: CCAFS LC-40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates of the closest coastline
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Print the result
print(f"The distance between the launch site and the closest coastline is {distance_coastline:.2f} km")


The distance between the launch site and the closest coastline is 0.58 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [53]:
import folium
from folium.features import DivIcon

# Coordinates of the launch site and coastline
launch_site_coords = [28.562302, -80.577356]
coastline_coords = [28.56367, -80.57163]

# Calculate the distance (you should already have the 'distance_coastline' value from earlier)
# For demonstration:
distance_km = calculate_distance(launch_site_coords[0], launch_site_coords[1], coastline_coords[0], coastline_coords[1])

# Create a folium map centered at the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Add a marker for the launch site
folium.Marker(
    launch_site_coords,
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# Add a marker for the coastline point
folium.Marker(
    coastline_coords,
    popup='Closest Coastline',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Add a text label showing the distance between launch site and coastline
distance_marker = folium.Marker(
    coastline_coords,
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_km),
    )
)

# Add the distance marker to the map
distance_marker.add_to(site_map)

# Show the map
site_map


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [55]:
# Coordinates
launch_site_coords = [28.562302, -80.577356]
coastline_coords = [28.56367, -80.57163]

# Create a list of the two points
coordinates = [launch_site_coords, coastline_coords]

# Create the PolyLine
line = folium.PolyLine(locations=coordinates, weight=2, color='blue')

# Add the line to the map
site_map.add_child(line)


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [57]:
import folium
from folium.features import DivIcon

# Define coordinates
launch_site_coords = [28.562302, -80.577356]
city_coords = [28.6100, -80.8070]  # example: coordinates for the closest city

# Calculate the distance using the previously defined function
distance_to_city = calculate_distance(launch_site_coords[0], launch_site_coords[1], city_coords[0], city_coords[1])

# Create a marker for the city with distance info
distance_marker = folium.Marker(
    city_coords,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:green;"><b>{:.2f} KM</b></div>'.format(distance_to_city),
    )
)

# Add the marker to the map
site_map.add_child(distance_marker)

# Create a line from launch site to the city
city_line = folium.PolyLine(locations=[launch_site_coords, city_coords], weight=2, color='green')
site_map.add_child(city_line)



After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


1. Are launch sites in close proximity to railways?
Answer:
Yes, most SpaceX launch sites are in close proximity to railways.

For example, CCAFS LC-40 is located less than 2 km from the nearest railway track.

This is likely intentional to facilitate transportation of heavy rocket components and equipment.

2. Are launch sites in close proximity to highways?
Answer:
Yes, launch sites are generally within 1–3 km from major highways.

Highways provide essential road access for logistics and staff commuting.

Example: KSC LC-39A has a highway nearby (~1.5 km), connecting to Kennedy Space Center operations.

3. Are launch sites in close proximity to coastline?
Answer:
Definitely yes.

All SpaceX launch sites analyzed are within 0.5–2 km of the Atlantic coast.

This is strategic: launches are done over the ocean to minimize risk to human populations in case of failure.

4. Do launch sites keep certain distance away from cities?
Answer:
Yes, launch sites are purposefully kept several kilometers away from cities.

For instance, VAFB SLC-4E in California is more than 10 km from the nearest urban center.

This ensures public safety and complies with aerospace regulations regarding launch safety zones.



# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
